### 라이브러리 및 데이터셋 호출

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt 

In [ ]:
df=pd.read_csv('../real_data/표준지공시지가_20230619.csv')

C:\Users\jeongbin\AppData\Local\Temp\ipykernel_24380\2064968684.py:1: DtypeWarning: Columns (4,5,23,24,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('../data/표준지공시지가_20230619.csv')


In [ ]:
df2=pd.read_csv('../real_data/표준지공시지가_20240101.csv')

C:\Users\jeongbin\AppData\Local\Temp\ipykernel_24380\4251090754.py:1: DtypeWarning: Columns (1,5,6,18,19,28,37,38,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df2=pd.read_csv('../data/표준지공시지가_20240101.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560000 entries, 0 to 559999
Data columns (total 31 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   일련번호          560000 non-null  int64  
 1   시군구           560000 non-null  int64  
 2   읍면동리          560000 non-null  int64  
 3   지번구분          560000 non-null  int64  
 4   본번지           560000 non-null  object 
 5   부번지           560000 non-null  object 
 6   시도명           560000 non-null  object 
 7   시군구명          557406 non-null  object 
 8   소재지           560000 non-null  object 
 9   지목            560000 non-null  object 
 10  면적            560000 non-null  float64
 11  용도지역1         560000 non-null  object 
 12  용도지역2         5719 non-null    object 
 13  이용상황          560000 non-null  object 
 14  주위환경          560000 non-null  object 
 15  지세명           560000 non-null  object 
 16  형상명           560000 non-null  object 
 17  도로교통          560000 non-null  object 
 18  공시지가

### 새로운 데이터셋 생성

23년도 데이터: x
24년도 공시지가값: y


In [ ]:
# 23년 데이터셋, 24년 데이터셋의 변수명 통일
df.rename(columns={'토지대장 번호(PNU)': '토지대장번호(PNU)'}, inplace=True)

In [6]:
# PNU 값에서 양쪽 공백 제거 (strip) 후 비교
df_pnu = df['토지대장번호(PNU)'].astype(str).str.strip()
df2_pnu = df2['토지대장번호(PNU)'].astype(str).str.strip()

# 공통 PNU 개수 계산
common_pnu_count = df_pnu.isin(df2_pnu).sum()

print(f"공통 PNU 개수: {common_pnu_count:,}개")


공통 PNU 개수: 465,769개


In [ ]:
# 1. df2에서 PNU와 공시지가만 추출해서 매핑용 테이블 생성
# '공시지가'를 '내년_공시지가'로 변경
pnu_price_map = df2[['토지대장번호(PNU)', '공시지가']].copy()
pnu_price_map.columns = ['토지대장번호(PNU)', '내년_공시지가']

# 2. PNU 공백 제거
df['토지대장번호(PNU)'] = df['토지대장번호(PNU)'].astype(str).str.strip()
pnu_price_map['토지대장번호(PNU)'] = pnu_price_map['토지대장번호(PNU)'].astype(str).str.strip()

# 3. df에 merge * 23년도 데이터셋 
df = df.merge(pnu_price_map, on='토지대장번호(PNU)', how='left')


In [ ]:
# 목표변수가 없는 행 제거 
df = df.dropna(subset=['내년_공시지가'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 465769 entries, 0 to 559999
Data columns (total 32 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   일련번호         465769 non-null  int64  
 1   시군구          465769 non-null  int64  
 2   읍면동리         465769 non-null  int64  
 3   지번구분         465769 non-null  int64  
 4   본번지          465769 non-null  object 
 5   부번지          465769 non-null  object 
 6   시도명          465769 non-null  object 
 7   시군구명         463255 non-null  object 
 8   소재지          465769 non-null  object 
 9   지목           465769 non-null  object 
 10  면적           465769 non-null  float64
 11  용도지역1        465769 non-null  object 
 12  용도지역2        4646 non-null    object 
 13  이용상황         465769 non-null  object 
 14  주위환경         465769 non-null  object 
 15  지세명          465769 non-null  object 
 16  형상명          465769 non-null  object 
 17  도로교통         465769 non-null  object 
 18  공시지가         465769 non-null 

### 변수 삭제 및 전처리

In [10]:
# 기존에 탐색을 통한 불필요 변수 삭제
df = df.drop(columns=[
    '일련번호', '토지대장번호(PNU)', '시군구', '읍면동리',
    '본번지', '부번지', '시군구명', '소재지',
    '공시지가', '지리적위치1','전년지가','용도지역2'
])


지번구분 전처리

In [11]:
# 1% 미만인 값들 묶기 
threshold=0.01
value_counts=df['지번구분'].value_counts(normalize=True)

# 1% 미만인 값들
rare_values=value_counts[value_counts < threshold].index

# 기타로 묶기
df['지번구분'] = df['지번구분'].apply(lambda x: '기타' if x in rare_values else x)

지목 전처리

In [12]:
# 1. 지목별 count 계산
jimok_counts = df['지목'].value_counts()

# 2. 기준에 따라 그룹화 (4000 미만은 '기타')
df['지목'] = df['지목'].apply(
    lambda x: x if jimok_counts[x] >= 4000 else '기타'
)


용도지역 재분류

In [13]:
replace_map = {
    '보전녹지지역': '자연녹지지역',
    '일반공업지역': '준공업지역',
    '준공업지역': '생산관리지역',
    '중심상업지역': '일반상업지역',
    '제1종전용주거지역': '제1종일반주거지역',
    '근린상업지역': '일반상업지역',
    '제2종전용주거지역': '제2종일반주거지역',
    '유통상업지역': '일반상업지역',
    '전용공업지역': '준공업지역',
    '용도미지정지역': '계획관리지역',
    '관리지역': '계획관리지역'
}

df['용도지역1'] = df['용도지역1'].replace(replace_map)
df['용도지역1'].value_counts()

용도지역1
계획관리지역       103557
농림지역          64811
제2종일반주거지역     61851
자연녹지지역        45540
생산관리지역        37142
보전관리지역        31452
제1종일반주거지역     31400
일반상업지역        29894
개발제한구역        17576
제3종일반주거지역     12916
준주거지역         10842
생산녹지지역         7354
자연환경보전지역       7238
준공업지역          4196
Name: count, dtype: int64

이용상황 재분류

In [19]:
# 통합 매핑 딕셔너리
replace_map_auto = {
    '다세대': '단독주택',
    '연립주택': '단독주택',
    '아파트': '단독주택',
    '주거기타': '단독주택',
    '주거나지': '단독주택',

    '상업기타': '상업용',
    '상업나지': '상업용',
    '업무용': '상업용',
    '휴게소': '상업용',
    '터미널': '상업용',
    '콘도': '상업용',
    '물류터미널': '상업용',
    '주차장등': '상업용',

    '주상기타': '주상용',
    '주상나지': '주상용',

    '공업기타': '공업용',
    '공업나지': '공업용',
    '발전소' :'공업용',
    '전기타': '전',
    '전축사': '전',
    '전창고': '전',

    '답기타': '답',
    '답창고': '답',
    '답축사': '답',
    
    '광천지': '임야기타',
    '염전': '임야기타',
    '양어·양식': '임야기타',
    '광업용지': '임야기타',

    '토지임야': '자연림',
    '조림': '자연림',
    '임야기타': '자연림',

    '골프장_대중제': '운동장등',
    '골프장_회원제': '운동장등',
    '골프장_간이': '운동장등',
    '승마장': '운동장등',
    '스키장': '운동장등',

    '유원지': '공원등',
    '하천등': '공원등',
    '공원묘지': '공원등',

    '태양광': '발전소',

    '유해혐오': '특수기타',
    '위험시설': '특수기타'
}

# 적용
df['이용상황'] = df['이용상황'].replace(replace_map_auto)
final_map = {
    '운동장등': '기타',
    '공원등': '기타',
    '목장용지': '기타',
    '특수기타': '기타',
    '공항': '기타',
    '도로등': '기타',
    '경마장': '기타',
    '기타시설': '기타'
}

# 적용
df['이용상황'] = df['이용상황'].replace(final_map)  
df['이용상황'].value_counts()


이용상황
단독주택    126941
전        98082
답        78285
자연림      55633
상업용      47454
주상용      31421
과수원      15423
공업용      11145
기타        1385
Name: count, dtype: int64

주위환경 재분류

In [20]:
df['주위환경'].value_counts()

주위환경
마을주변 농경지대      58213
기존 주택지대        32719
순수 농경지대        29380
산간 농경지대        28412
주택 및 상가혼용지대    23692
               ...  
온천 휴양지대           11
온천지대               8
온천 관광지대            6
해안 목장지대            5
계곡 유원지대            1
Name: count, Length: 107, dtype: int64

In [21]:
mapping = {}

for env in df['주위환경']:
    if '농경' in env or '경지정리' in env or '농촌' in env:
        mapping[env] = '농촌·경작지대'
    elif '주택' in env or '단지' in env or '아파트' in env or '연립' in env or '전원' in env or '한옥' in env:
        mapping[env] = '주거지대'
    elif '상가' in env or '시장' in env or '업무' in env:
        mapping[env] = '상업지대'
    elif '산림' in env or '야산' in env or '임야' in env:
        mapping[env] = '산림지대'
    elif '공장' in env or '산업단지' in env or '농공단지' in env:
        mapping[env] = '공업지대'
    elif '취락' in env or '마을' in env:
        mapping[env] = '취락지대'
    elif '해안' in env:
        mapping[env] = '해안지대'
    elif '유원지' in env or '관광' in env or '온천' in env:
        mapping[env] = '관광·휴양지대'
    elif '도로' in env or '노선' in env or '역' in env:
        mapping[env] = '교통연계지대'
    elif '재개발' in env or '예정' in env:
        mapping[env] = '개발예정지대'
    elif '기타' in env or '미개발' in env:
        mapping[env] = '기타'
    else:
        mapping[env] = '기타'
df['주위환경'] = df['주위환경'].replace(mapping)
df['주위환경'].value_counts()

주위환경
농촌·경작지대    239896
주거지대       103474
산림지대        55822
상업지대        50821
공업지대         8328
취락지대         2395
개발예정지대       2278
기타           1990
해안지대          430
관광·휴양지대       335
Name: count, dtype: int64

In [24]:
# 1% 미만인 값들 묶기 
threshold=0.01
value_counts=df['주위환경'].value_counts(normalize=True)

# 1% 미만인 값들
rare_values=value_counts[value_counts < threshold].index

# 기타로 묶기
df['주위환경'] = df['주위환경'].apply(lambda x: '기타' if x in rare_values else x)
df['주위환경'].value_counts()

주위환경
농촌·경작지대    239896
주거지대       103474
산림지대        55822
상업지대        50821
공업지대         8328
기타           7428
Name: count, dtype: int64

지세명 전처리

In [26]:
고_저지대=['고지','저지']
df['지세명']=df['지세명'].replace(고_저지대,'고,저지대')
df['지세명'].value_counts()

지세명
평지       283798
완경사지     152411
급경사지      26937
고,저지대      2623
Name: count, dtype: int64

지리적 위치 제거

In [29]:
df['지리적위치2'].value_counts()

지리적위치2
내               53767
남측인근            32146
북측인근            30027
동측인근            28445
서측인근            27841
                ...  
자연부락내               1
북측경사면               1
남동측경사면              1
남동측인근(새마을금고)        1
북서측경사면              1
Name: count, Length: 1864, dtype: int64

In [30]:
df = df.drop(['지리적위치2'], axis=1)

방위 제거

In [31]:
df['방위'].value_counts()

방위
남향     34555
동향     26707
남동향    26189
남서향    24347
서향     23455
북향     17788
북동향    15404
북서향    14124
Name: count, dtype: int64

In [32]:
df = df.drop(['방위'], axis=1)

용도지구 재분류

In [33]:
df['용도지구'] = df['용도지구'].fillna('미지정')
df['용도지구'].value_counts()

용도지구
미지정                 405027
자연취락지구               22832
방화지구                 10004
고도지구                  8107
시가지경관지구               7499
주거개발진흥지구              2726
특화경관지구                2302
중요시설물(공항)보호지구         1617
기타지구                  1432
자연경관지구                1105
관광휴양개발진흥지구             634
집단취락지구                 621
특정개발진흥지구               395
기타경관지구                 360
산업유통개발진흥지구             343
역사문화환경보호지구             172
중요시설물(항만)보호지구          143
특정용도제한지구               140
방재지구                   122
생태계보호지구                 61
중요시설물(교정.군사)보호지구        39
복합개발진흥지구                29
복합용도지구                  26
중요시설물(공용시설)보호지구         18
중요시설물보호지구               15
Name: count, dtype: int64

In [36]:
replace_map = {
    '주거개발진흥지구': '자연취락지구',
    '집단취락지구': '자연취락지구',
    '관광휴양개발진흥지구': '자연경관지구',
    '산업유통개발진흥지구': '기타지구',
    '특정개발진흥지구': '기타지구',
    '복합개발진흥지구': '기타지구',
    '복합용도지구': '기타지구',
    '기타경관지구': '시가지경관지구',
    '특화경관지구': '시가지경관지구',
    '자연경관지구': '시가지경관지구',
    '역사문화환경보호지구': '시가지경관지구',
    '방재지구': '방화지구',
    '생태계보호지구': '자연경관지구',
    '특정용도제한지구': '기타지구',
    '중요시설물(공항)보호지구': '중요시설물보호지구',
    '중요시설물(항만)보호지구': '중요시설물보호지구',
    '중요시설물(공용시설)보호지구': '중요시설물보호지구',
    '중요시설물(교정.군사)보호지구': '중요시설물보호지구',
}

df['용도지구'] = df['용도지구'].replace(replace_map)
replace_map={'중요시설물보호지구':'중요시설물보호및기타지구',
            '기타지구':'중요시설물보호및기타지구'}
df['용도지구'] = df['용도지구'].replace(replace_map)
df['용도지구'].value_counts()


용도지구
미지정             405027
자연취락지구           26179
시가지경관지구          12133
방화지구             10126
고도지구              8107
중요시설물보호및기타지구      4197
Name: count, dtype: int64

농지구분, 임야구분, 경지정리가 서로 유사한 변수인지 판단

In [37]:
def null_combo_report(df, col1, col2):
    total = len(df)

    both_notnull = df[col1].notnull() & df[col2].notnull()
    one_null = df[col1].isnull() ^ df[col2].isnull()
    both_null = df[col1].isnull() & df[col2].isnull()

    print(f"✅ '{col1}' & '{col2}' 조합 분석")
    print(f"- 둘 다 값 있음: {both_notnull.sum():,}개 ({both_notnull.mean():.2%})")
    print(f"- 하나만 값 있음: {one_null.sum():,}개 ({one_null.mean():.2%})")
    print(f"- 둘 다 결측치: {both_null.sum():,}개 ({both_null.mean():.2%})\n")

# 적용
null_combo_report(df, '농지구분', '임야구분')
null_combo_report(df, '농지구분', '경지정리')
null_combo_report(df, '임야구분', '경지정리')

✅ '농지구분' & '임야구분' 조합 분석
- 둘 다 값 있음: 0개 (0.00%)
- 하나만 값 있음: 105,122개 (22.57%)
- 둘 다 결측치: 360,647개 (77.43%)

✅ '농지구분' & '경지정리' 조합 분석
- 둘 다 값 있음: 49,493개 (10.63%)
- 하나만 값 있음: 707개 (0.15%)
- 둘 다 결측치: 415,569개 (89.22%)

✅ '임야구분' & '경지정리' 조합 분석
- 둘 다 값 있음: 0개 (0.00%)
- 하나만 값 있음: 105,823개 (22.72%)
- 둘 다 결측치: 359,946개 (77.28%)



In [38]:
df['농지구분'] = df['농지구분'].fillna('비농지')
df['임야구분'] = df['임야구분'].fillna('비임야')
df = df.drop(['경지정리'], axis=1)

기타제한 1 전처리

In [41]:
# 기타제한1 최빈값으로 결측치 대체
mode_기타제한 = df['기타제한'].mode()[0]
df['기타제한'] = df['기타제한'].fillna(mode_기타제한)
replace_map = {
    '수변구역': '상수원보호구역',
    '수산자원보호구역': '상수원보호구역 기타',
    '공원자연환경지구': '공원구역',
    '공원자연보존지구': '공원구역',
    '공원마을지구': '공원구역',
    '공원문화유산지구': '공원구역',
    '도시자연공원구역': '공원구역',
    '홍수관리구역': '하천구역',
    '전원개발사업구역': '허가구역',
    '농공단지': '허가구역',
    '고속접도구역': '일반접도구역',
    '공항소음대책지역': '군사기지및군사시설보호구역,비행안전구역',
    '비오톱': '공원구역',
    '시가화조정구역': '지구단위계획구역',
    '일시적규제지역': '특별대책지역',
    '친수구역': '상수원보호구역',
}

df['기타제한'] = df['기타제한'].replace(replace_map)
df['기타제한'].value_counts()

기타제한
기타                      269758
교육환경보호구역                 62150
지구단위계획구역                 40146
군사기지및군사시설보호구역,비행안전구역     19538
상수원보호구역 기타               14992
특별대책지역                   12437
문화재보호구역                  10701
하천구역                      9288
일반접도구역                    8821
허가구역                      7541
상수원보호구역                   6608
공원구역                      3789
Name: count, dtype: int64

계획시설 제거

In [42]:
df = df.drop(['계획시설'], axis=1)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 465769 entries, 0 to 559999
Data columns (total 16 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   지번구분       465769 non-null  object 
 1   시도명        465769 non-null  object 
 2   지목         465769 non-null  object 
 3   면적         465769 non-null  float64
 4   용도지역1      465769 non-null  object 
 5   이용상황       465769 non-null  object 
 6   주위환경       465769 non-null  object 
 7   지세명        465769 non-null  object 
 8   형상명        465769 non-null  object 
 9   도로교통       465769 non-null  object 
 10  용도지구       465769 non-null  object 
 11  농지구분       465769 non-null  object 
 12  기타제한       465769 non-null  object 
 13  계획시설 저촉여부  465769 non-null  object 
 14  임야구분       465769 non-null  object 
 15  내년_공시지가    465769 non-null  float64
dtypes: float64(2), object(14)
memory usage: 60.4+ MB


전처리 데이터 저장

In [45]:
df.to_csv("../real_data/전처리 데이터.csv", index=False, encoding='utf-8-sig')
